In [45]:
import numpy as np
import pandas as pd
import pandas_profiling
import math
import re
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.impute import SimpleImputer
from spellchecker import SpellChecker
from word2number import w2n

In [2]:
def is_number(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
spell = SpellChecker()

### Merge Duplicate data in train

In [4]:
def merge(grp):
    df = pd.DataFrame()
    if(grp.shape[0] > 1):
        for c in grp.columns:
            value_counts = grp[c].value_counts().index.astype(grp[c].dtypes)
            if value_counts.size > 1:
                print(grp, value_counts) #Error
            elif value_counts.size == 1:
                df[c] = value_counts[0]
            else:
                df[c] = None
    else:
        df = grp.head(1)
    return df
train = train.groupby("id").apply(merge)

In [5]:
ntrain = train.shape[0]
ntest = train.shape[0]
all_data = pd.concat((train, test)).reset_index(drop=True)
all_data.drop(['stroke_in_2018'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

all_data size is : (43342, 13)


C:\Users\thien\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


### Data Cleaning

In [6]:
def is_job_status_and_living_area_reversed(x):
    return ((x["job_status"] != None and x["job_status"] in ("r", "c", "city", "remote", "remotee"))
             or (x["living_area"] != None and x["living_area"] in ("private_sector", "business_owner")))
def process_job_status(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x in ("private sector", "privattte", "private", "private_sector"):
        return "private_sector"
    elif x in ("government", "govt."):
        return "government"
    elif x in ("business_owner", "business owner", "biz"):
        return "business_owner"
    elif x in ("parental_leave", "parental leave"):
        return "parental_leave"
    else:
        return x
    
def process_living_area(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x == 'c':
        return 'city'
    elif x in ('r', 'remotee'):
        return 'remote'
    else:
        return x

def split_job_status_and_living_area(x):
    pair = x.lower().split("?") if x != None else [x, x]
    if len(pair) < 2:
        pair = [pair[0], None]
    return pair

def process_job_status_and_living_area(df):
    df["job_status"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[0])
    df["living_area"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[1])
    job_status = df.apply(lambda x: x["living_area"] if is_job_status_and_living_area_reversed(x) else x["job_status"], 1)
    living_area = df.apply(lambda x: x["job_status"] if is_job_status_and_living_area_reversed(x) else x["living_area"], 1)
    df["job_status"] = job_status.apply(lambda x: process_job_status(x))
    df["living_area"] = living_area.apply(lambda x: process_living_area(x))
    df.drop(columns='job_status and living_area',inplace=True)

In [7]:
process_job_status_and_living_area(all_data)
#process_job_status_and_living_area(test)

In [8]:
def process_smoker_status(x):
    if x == None:
        return None
    elif x.startswith("non"):
        return "non-smoker"
    elif x.startswith("quit"):
        return "quit"
    elif x.startswith("active"):
        return "active_smoker"
    else:
        return None
all_data["smoker_status"] = all_data["smoker_status"].astype(str).apply(process_smoker_status)
#test["smoker_status"] = test["smoker_status"].astype(str).apply(process_smoker_status)

In [9]:
def process_binary_col(df,columns):
    for col in columns:
        df[col] = pd.to_numeric(df[col],errors="coerce")
        df[col] = df[col].astype(int,errors='ignore')
        df[col] = df[col].apply(lambda x: x if x in [0,1] else None)

In [10]:
process_binary_col(all_data,["heart_condition_detected_2017","high_BP","married"])

Convert BMI to numeric

In [11]:
all_data["BMI"] = pd.to_numeric(all_data["BMI"],errors="coerce")

In [12]:
# Process gender into oneof "F", "M" and "OTHER"
def genderSpellingRewrite(gender_str):
    if not isinstance(gender_str, str):
        return None;
    uppered = gender_str.upper()
    # Repeated single occurence should be truncate.
    patternM = re.compile('[M]+$')
    if (patternM.match(uppered)):
        return "M"
    patternF = re.compile('[F]+$')
    if (patternF.match(uppered)):
        return "F"
    # Misspelling should be corrected and replaced.
    # TODO: Malle is not going to be corrected as Male.Need to update spell's known list.
    corrected = spell.correction(uppered).upper()
    if (corrected == "FEMALE"):
        return "F"
    if (corrected == "MALE"):
        return "M"
    if (corrected == "OTHER"):
        return "OTHER"
    return None

In [13]:
# Process human number word into number
def numberConversion(potential_number_word):
    # Correct any possible miss spelled number_word
    corrected_potential_word = spell.correction(potential_number_word)
    # check it it means number
    try:
      potential_num = w2n.word_to_num(corrected_potential_word)
    except ValueError:
        return potential_number_word
    return potential_num

In [14]:
def formatSexAge(origin_str):
    if not isinstance(origin_str, str):
        return [None,None]
    # Preprocess 
    # Entry with missing column.
    if (',' not in origin_str):
        origin_str = origin_str + ',';
    origin_list = origin_str.replace(" ", "").upper().split(",")
    if(origin_list[0].upper() == "NAN"):
        origin_list[0] = ""
    if(origin_list[1].upper() == "NAN"):
        origin_list[1] = ""
    # Convert possible number in entry.
    if((not is_number(origin_list[0])) and (not is_number(origin_list[1]))):
        origin_list[0] = numberConversion(origin_list[0])
        origin_list[1] = numberConversion(origin_list[1])
    genderSet = set(['F', 'M', 'OTHER'])
    if (is_number(origin_list[0])):
        # wrong entry (num, num)
        if (is_number(origin_list[1])):
            if (origin_list[0] == origin_list[1]):
                return [None, origin_list[0]]
            return [None, None]
        else: # first number, second '' or gender (NOT num for sure)
          # swap back number
          origin_list = origin_list[::-1]
          origin_list[0] = genderSpellingRewrite(origin_list[0])
          return origin_list
    else: 
        origin_list[0] = genderSpellingRewrite(origin_list[0])
         # first '' or str, second is number
        if (is_number(origin_list[1])):
            return origin_list
        else:
            origin_list[1] = genderSpellingRewrite(origin_list[1])
            if(origin_list[0] == origin_list[1]):
               origin_list[1] = None
            return origin_list

In [15]:
list(all_data)

['BMI',
 'TreatmentA',
 'TreatmentB',
 'TreatmentC',
 'TreatmentD',
 'average_blood_sugar',
 'heart_condition_detected_2017',
 'high_BP',
 'id',
 'married',
 'sex and age',
 'smoker_status',
 'job_status',
 'living_area']

In [16]:
def process_sex_age_(df):
    df["sex_age_list"] = df["sex and age"].astype(str).apply(lambda x: formatSexAge(x))
    df[['sex','age']] = pd.DataFrame(df["sex_age_list"].values.tolist(), index= df.index)
    df[df["sex"] == "None"]["sex"] = "OTHER"
process_sex_age_(all_data)

Clean Sex and Age

In [17]:
# def clean_sex_age(sex_age_list):
#     if type(sex_age_list) is not list:
#         return [None,None]
#     # Strip and Upper case both sex and age
#     sex_age_list[0],sex_age_list[1] = sex_age_list[0].strip().upper(), sex_age_list[1].strip().upper()
    
#     # 2nd : first one is empty and second one is not numeric
#     if (is_number(sex_age_list[0]) or (not sex_age_list[0] and not is_number(sex_age_list[1]))): 
#         sex_age_list = sex_age_list[::-1]
#     sex = sex_age_list[0].strip().upper()
    
#     if sex in ('FEMALE','FEMALLE'):
#         sex = 'F'
#     if sex in ('MALE','MMALE','MM'):
#         sex = 'M'

#     sex_age_list[0] = sex
#     sex_age_list[1] = sex_age_list[1]
#     return sex_age_list

# def process_sex_age(df):
#     df["sex_age_list"] = df["sex and age"].str.split(",").apply(clean_sex_age)
#     df[['sex','age']] = pd.DataFrame(df["sex_age_list"].values.tolist(), index= df.index)
#     df["age"] = pd.to_numeric(df["age"],errors='coerce').round()
#     return df.drop(columns=["sex_age_list","sex and age"])
# all_data = process_sex_age(all_data)

In [18]:
all_data["age"] = pd.to_numeric(all_data["age"],errors="coerce")

In [19]:
#all_data.drop_duplicates(inplace=True)

In [20]:
def split_train_test(df):
    return df[:ntrain], df[ntrain:]

In [21]:
preprocessed_train, preprocessed_test = split_train_test(all_data)

### Missing Values

In [22]:
def impute_by_mode(df,columns):
    for col in columns:
        df[col] = df[col].fillna(df[col].mode().iloc[0])

In [23]:
impute_by_mode(all_data,["heart_condition_detected_2017", "high_BP", "married", "job_status"])

In [24]:
all_data["sex"] = all_data["sex"].fillna("OTHER").astype(str)

Impute Age by Median

In [25]:
df_tmp = preprocessed_train.groupby("sex")["age"].median().reset_index(name="MedianAge")
df_merge = all_data.merge(df_tmp,on="sex",how="left")
cond = df_merge['age'].isnull()
df_merge['age'] = df_merge['age'].fillna(df_merge["MedianAge"])
all_data = df_merge.drop(columns="MedianAge")

In [26]:
all_data["married"] = all_data["married"].astype(bool)
all_data["heart_condition_detected_2017"] = all_data["heart_condition_detected_2017"].astype(bool)
all_data["high_BP"] = all_data["high_BP"].astype(bool)

all_data.dtypes

BMI                              float64
TreatmentA                       float64
TreatmentB                       float64
TreatmentC                       float64
TreatmentD                        object
average_blood_sugar              float64
heart_condition_detected_2017       bool
high_BP                             bool
id                                 int64
married                             bool
sex and age                       object
smoker_status                     object
job_status                        object
living_area                       object
sex_age_list                      object
sex                               object
age                              float64
dtype: object

Check missing value of job_status

In [27]:
preprocessed_train, preprocessed_test = split_train_test(all_data)

In [43]:
def rf_predict_missing(df,missing_var,independent_var):
    non_missing_data = df[df[missing_var].notnull()]
    
    # Remove missing values
#     non_missing_data = non_missing_data.dropna()
    
    # Build Random Forest classifier
    clf = make_pipeline(SimpleImputer(strategy = "most_frequent"), OneHotEncoder(handle_unknown="ignore"),AdaBoostClassifier())
    clf.fit(non_missing_data[independent_var],non_missing_data[missing_var])
    acc = np.mean(cross_val_score(clf,non_missing_data[independent_var],non_missing_data[missing_var],cv=10))
    print(cross_val_score(clf,non_missing_data[independent_var],non_missing_data[missing_var],cv=10))
    print("Random Forest Mean Accuracy for 5 runs of cross validation ", acc)
    
    # Predict missing values
#     cond = df[missing_var].isnull()
#     df[cond][missing_var] = clf.predict(df[cond][independent_var])
    return clf

In [29]:
all_data["job_status"].value_counts()

private_sector    24807
business_owner     6783
parental_leave     6144
government         5432
unemployed          176
Name: job_status, dtype: int64

In [30]:
all_data.shape

(43342, 17)

In [31]:
# clf = rf_predict_missing(all_data,"job_status",["BMI","age","sex","married","high_BP"])

In [32]:
# clf.predict(all_data[all_data["job_status"].isnull()][["BMI", "age", "sex", "married", "high_BP"]])

NameError: name 'clf' is not defined

In [33]:
profile = all_data.profile_report(title='Medical Record Profiling Report')
profile.to_file(output_file="train_data_summary.html")

In [34]:
def bmi_category(bmi):
    if pd.isnull(bmi):
        return np.nan
    if bmi > 35:
        return "SO"
    elif bmi > 30:
        return "MO"
    elif bmi > 25:
        return "O"
    elif bmi > 18.5:
        return "N"
    else:
        return "U"

In [35]:
all_data["bmi_category"] = all_data["BMI"].apply(bmi_category)

In [36]:
all_data["bmi_category"].value_counts()

O     11459
SO    10662
MO    10269
N      8354
U      1118
Name: bmi_category, dtype: int64

In [37]:
list(all_data)

['BMI',
 'TreatmentA',
 'TreatmentB',
 'TreatmentC',
 'TreatmentD',
 'average_blood_sugar',
 'heart_condition_detected_2017',
 'high_BP',
 'id',
 'married',
 'sex_and_age',
 'smoker_status',
 'job_status',
 'living_area',
 'sex_age_list',
 'sex',
 'age',
 'bmi_category']

In [38]:
all_data[~all_data["bmi_category"].isnull()]["bmi_category"].value_counts()

O     11459
SO    10662
MO    10269
N      8354
U      1118
Name: bmi_category, dtype: int64

In [46]:
rf_predict_missing(all_data,"bmi_category", ["job_status", "high_BP", "heart_condition_detected_2017", "married", "age", "sex", "average_blood_sugar"])

[0.37702961 0.39350525 0.38046334 0.38619537 0.38509317 0.37195413
 0.36813187 0.38198758 0.38661888 0.38106622]
Random Forest Mean Accuracy for 5 runs of cross validation  0.3813718508053037


Pipeline(memory=None,
     steps=[('simpleimputer', SimpleImputer(copy=True, fill_value=None, missing_values=nan,
       strategy='most_frequent', verbose=0)), ('onehotencoder', OneHotEncoder(categorical_features=None, categories=None,
       dtype=<class 'numpy.float64'>, handle_unknown='ignore',
       n_values=None, sparse=True)), ('adaboostclassifier', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None))])

In [40]:
all_data[all_data["sex"] != "OTHER"]["sex"].value_counts()

F    25635
M    17682
Name: sex, dtype: int64

In [ ]:
all_Data

In [ ]:
all_data["sex"]